![alt text](asset/shanaly_Ai.jpg)

In [1]:
# import all library that will used 
import torch
import matplotlib
import os
import sklearn
import sys
import PIL
import torchvision

In [2]:
# Print out the version of library 
print(sys.platform)
print(f"torchvision version {torchvision.__version__}")
print(f"Pillow version {PIL.__version__}")
print(f"torch version {torch.__version__} ")
print(f"matplotlib version {matplotlib.__version__}")
print(f"Sklearn version {sklearn.__version__}")

win32
torchvision version 0.20.1+cpu
Pillow version 11.0.0
torch version 2.5.1+cpu 
matplotlib version 3.9.2
Sklearn version 1.5.2


In [3]:
# import all important class and module used 
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from sklearn.datasets import load_digits
import torch.utils.data as Data
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

In [4]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.out = nn.Linear(32*2*2, 10)  

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        flatten = nn.Flatten()
        x = flatten(x)
        output = self.out(x)
        return output



In [5]:
# Initialize model
model = CNN()
model

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=128, out_features=10, bias=True)
)

In [6]:
# Load dataset
digits = load_digits()
images = digits.images
target = digits.target
tensor_images = torch.from_numpy(images).float()
print(tensor_images.shape)

torch.Size([1797, 8, 8])


In [7]:
# Prepare Dataset
class Dataset(Data.Dataset):
    def __init__(self, images, target):
        self.images = images
        self.target = target

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        target = self.target[index]
        image = image.unsqueeze(0)  
        return (image, target)

dataset = Dataset(tensor_images, target)
train_loader = Data.DataLoader(dataset=dataset, batch_size=8, shuffle=True)

print(dataset[1][0].shape) # the reason we squueze is to made a 3D with [channel, length, width ]

torch.Size([1, 8, 8])


In [8]:
train_loader.dataset[0] # example of one data contain image and its target

(tensor([[[ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.],
          [ 0.,  0., 13., 15., 10., 15.,  5.,  0.],
          [ 0.,  3., 15.,  2.,  0., 11.,  8.,  0.],
          [ 0.,  4., 12.,  0.,  0.,  8.,  8.,  0.],
          [ 0.,  5.,  8.,  0.,  0.,  9.,  8.,  0.],
          [ 0.,  4., 11.,  0.,  1., 12.,  7.,  0.],
          [ 0.,  2., 14.,  5., 10., 12.,  0.,  0.],
          [ 0.,  0.,  6., 13., 10.,  0.,  0.,  0.]]]),
 np.int64(0))

In [9]:
# Hyperparameters
EPOCHS = 50
Lr = 0.001

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)

In [10]:
def train(epochs, model, dataloader):
    # Training the model
    total_loss = 0.0
    model.train()
    for epoch in range(epochs):
        for img, target in tqdm(dataloader, desc="Training", leave=False):
            optimizer.zero_grad()  
            output = model(img)
            loss = criterion(output, target)  
            loss.backward()
            optimizer.step()  
            total_loss += loss.item()
        
        print(f"Loss = {total_loss/len(dataloader.dataset)}")

In [42]:
train(1, model, train_loader)

Training:   0%|          | 0/225 [00:00<?, ?it/s]

Loss = 0.001117730934734733


In [43]:
def predict(model, data_loader, device="cpu"):
    all_probs = torch.tensor([]).to(device)
    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, desc="Predicting", leave=False):
            inputs = inputs.to(device)
            output = model(inputs)
            probs = F.softmax(output, dim=1)
            all_probs = torch.cat((all_probs, probs), dim=0)

    return all_probs

In [44]:
probabilities_train = predict(model, train_loader)
probabilities_train.shape

Predicting:   0%|          | 0/225 [00:00<?, ?it/s]

torch.Size([1797, 10])

In [45]:
def score(model, data_loader, loss_fn, device="cpu"):
    
    total_loss = 0
    total_correct = 0

    model.eval()

  
    with torch.no_grad():
        
        for inputs, targets in tqdm(data_loader, desc="Scoring", leave=False):
            inputs = inputs.to(device)
            output = model(inputs)

            targets = targets.to(device)
            loss = loss_fn(output, targets)
            total_loss += loss.data.item() * inputs.size(0)

            correct = torch.eq(torch.argmax(output, dim=1), targets)
            total_correct += torch.sum(correct).item()

    return total_loss / len(data_loader.dataset), total_correct / len(
        data_loader.dataset
    )

In [47]:
loss_train, accuracy_train = score(model, train_loader, criterion)
print(f"Training accuracy from score function: {accuracy_train:.2f} and loss is {loss_train:.2f}")

Scoring:   0%|          | 0/225 [00:00<?, ?it/s]

Training accuracy from score function: 1.00 and loss is 0.00


*Shanaly AI @ Alfred Baraka Rugoye*